# corona 데이터 로드
1. 데이터 로드
2. unnamed: 0, seq, stateDt, statetime, updateDt 컬럼 삭제
3. 인덱스의 값을 createDt 변경 (시계열로 데이터를 변환하여 인덱스로 이동)
4. 컬럼의 이름 변경 ['사망자','확진자','누적의심자','누적확진율']
5. 인덱스 이름 변경 : 등록일시
6. 일일확진자, 일일 사망자 컬럼 추가 / 사망자, 확진자의 차이를 대입 / 결측치는 0으로 대체
7. 누적의심자, 누적확진율에 결측치 데이터는 전의 데이터값으로 대체
8. 인덱스를 기준으로 오름차순 정렬 변경
9. 일일확진자가 음수인 값을 찾아서 인덱스를 삭제

In [1]:
import pandas as pd

In [47]:
corona = pd.read_csv('../csv/corona.csv')
corona.head(1)

,Unnamed: 0,createDt,deathCnt,decideCnt,seq,stateDt,stateTime,updateDt,accExamCnt,accDefRate
0,0,2022-06-08 09:09:05.982,24305,18188200,904,20220608,00:00,NaN,NaN,NaN


In [48]:
corona = corona.drop(['Unnamed: 0','stateDt', 'stateTime', 'updateDt'], axis=1)

In [49]:
corona = corona.drop(['seq'], axis=1)

In [50]:
corona.head(1)

,createDt,deathCnt,decideCnt,accExamCnt,accDefRate
0,2022-06-08 09:09:05.982,24305,18188200,NaN,NaN


In [51]:
corona.columns = ['등록일시','사망자','확진자','누적의심자','누적확진율']

In [52]:
corona['등록일시'] = pd.to_datetime(corona['등록일시'])

In [53]:
corona = corona.set_index('등록일시')

In [54]:
corona['일일확진자'] = corona['확진자'] - corona['확진자'].shift(-1).fillna(0)

In [55]:
corona['일일사망자'] = corona['사망자'] - corona['사망자'].shift(-1).fillna(0)

In [56]:
corona.head(5)

,사망자,확진자,누적의심자,누적확진율,일일확진자,일일사망자
등록일시,,,,,,
2022-06-08 09:09:05.982,24305,18188200,NaN,NaN,13358.0,6.0
2022-06-07 09:09:00.897,24299,18174842,NaN,NaN,6172.0,20.0
2022-06-06 09:00:06.734,24279,18168670,NaN,NaN,5022.0,21.0
2022-06-05 08:53:19.426,24258,18163648,NaN,NaN,9834.0,20.0
2022-06-04 08:56:49.219,24238,18153814,NaN,NaN,12039.0,9.0


In [57]:
corona['누적의심자'] = corona['누적의심자'].fillna(method='bfill')

In [58]:
corona['누적확진율'] = corona['누적확진율'].fillna(method='bfill')

In [59]:
corona.head()

,사망자,확진자,누적의심자,누적확진율,일일확진자,일일사망자
등록일시,,,,,,
2022-06-08 09:09:05.982,24305,18188200,21518073.0,2.787908,13358.0,6.0
2022-06-07 09:09:00.897,24299,18174842,21518073.0,2.787908,6172.0,20.0
2022-06-06 09:00:06.734,24279,18168670,21518073.0,2.787908,5022.0,21.0
2022-06-05 08:53:19.426,24258,18163648,21518073.0,2.787908,9834.0,20.0
2022-06-04 08:56:49.219,24238,18153814,21518073.0,2.787908,12039.0,9.0


In [60]:
corona = corona.sort_index()

In [61]:
corona.head()

,사망자,확진자,누적의심자,누적확진율,일일확진자,일일사망자
등록일시,,,,,,
2020-03-10,54,7513,210144.0,3.919308,7513.0,54.0
2020-03-11,60,7755,222395.0,3.804175,242.0,6.0
2020-03-12,66,7869,234998.0,3.621744,114.0,6.0
2020-03-13,67,7979,248647.0,3.458499,110.0,1.0
2020-03-14,72,8086,261335.0,3.318000,107.0,5.0


# another type of way

In [39]:
# 1 ~ 3번까지의 작업 한번에 하기
corona1 = pd.read_csv('../csv/corona.csv', usecols=[1,2,3,7,8], parse_dates=[0], index_col=[0])

In [40]:
corona1.head(1)

,deathCnt,decideCnt,updateDt,accExamCnt
createDt,,,,
2022-06-08 09:09:05.982,24305,18188200,NaN,NaN


In [41]:
corona1.index.name = '등록일시'
corona1.columns = ['확진자','사망자','누적의심자','누적확진율']
corona1.head(1)

,확진자,사망자,누적의심자,누적확진율
등록일시,,,,
2022-06-08 09:09:05.982,24305,18188200,NaN,NaN


In [42]:
corona1 = corona1.fillna(method='bfill')

In [43]:
corona1

,확진자,사망자,누적의심자,누적확진율
등록일시,,,,
2022-06-08 09:09:05.982,24305,18188200,2022-06-08 09:10:36.846,21518073.0
2022-06-07 09:09:00.897,24299,18174842,2022-06-08 09:10:36.846,21518073.0
2022-06-06 09:00:06.734,24279,18168670,2022-06-08 09:10:50.441,21518073.0
2022-06-05 08:53:19.426,24258,18163648,2022-06-08 09:11:04.758,21518073.0
2022-06-04 08:56:49.219,24238,18153814,2022-06-08 09:11:26.303,21518073.0
...,...,...,...,...
2020-03-14 00:00:00.000,72,8086,2021-10-07 10:30:51.51,261335.0
2020-03-13 00:00:00.000,67,7979,2021-10-07 10:30:51.51,248647.0
2020-03-12 00:00:00.000,66,7869,2021-10-07 10:30:51.51,234998.0


In [45]:
corona1['일일확진자'] = corona1['확진자'] - corona1['확진자'].diff(-1).fillna(0)
corona1['일일사망자'] = corona1['사망자'] - corona1['사망자'].shift(-1).fillna(0)

In [46]:
corona1.sort_index(inplace=True)

In [63]:
corona1.head()

,확진자,사망자,누적의심자,누적확진율,일일확진자,일일사망자
등록일시,,,,,,
2020-03-10,54,7513,2021-10-07 10:30:51.51,210144.0,54.0,7513.0
2020-03-11,60,7755,2021-10-07 10:30:51.51,222395.0,54.0,242.0
2020-03-12,66,7869,2021-10-07 10:30:51.51,234998.0,60.0,114.0
2020-03-13,67,7979,2021-10-07 10:30:51.51,248647.0,66.0,110.0
2020-03-14,72,8086,2021-10-07 10:30:51.51,261335.0,67.0,107.0


In [62]:
corona1.query('일일확진자 < 0')

,확진자,사망자,누적의심자,누적확진율,일일확진자,일일사망자
등록일시,,,,,,
